In [ ]:
##--------------------------tunned CNN Model + Attention--------------------------

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import time

import tensorflow as tf
from datetime import datetime

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, LSTM
from tensorflow.keras.layers import Dense, Dropout, Flatten, Multiply, Permute, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K

# === Load datasets ===
attackFree= pd.read_csv('Attack_free new.csv')[0:2369397]
doS= pd.read_csv('DoS_Attack_new.csv')[0:656578]
fuzzy= pd.read_csv('Fuzzy_Attack_New.csv')[0:591989]
impersonation= pd.read_csv('Impersonation_Attack_New.csv')[0:995471]

# === Add labels ===
for df, label in zip([attack_free, dos, fuzzy, impersonation], [0, 1, 2, 3]):
    df['label'] = label

df = pd.concat([attack_free, dos, fuzzy, impersonation], ignore_index=True)
df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])

# === Convert HEX to numeric ===
for col in df.columns[:-1]:
    df[col] = df[col].apply(lambda x: int(str(x), 16) if isinstance(x, str) and all(c in '0123456789abcdefABCDEF' for c in str(x)) else x)
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(inplace=True)

# === Feature matrix and labels ===
X = df.drop(columns='label').values
y = df['label'].values

# === Normalize ===
scaler = StandardScaler()
X = scaler.fit_transform(X)

# === Reshape for Conv1D ===
X = X.reshape(X.shape[0], X.shape[1], 1)

# === Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)
y_train_cat = to_categorical(y_train, num_classes=4)
y_test_cat = to_categorical(y_test, num_classes=4)

# === Compute class weights ===
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

# === Custom Attention ===
def attention_layer(inputs):
    attention = Dense(inputs.shape[1], activation='softmax')(inputs)
    attention = Permute((2, 1))(attention)
    attention = Lambda(lambda x: K.mean(x, axis=1))(attention)
    attention = Dense(inputs.shape[-1], activation='sigmoid')(attention)
    return Multiply()([inputs, attention])

# === Build Model ===
input_layer = Input(shape=(X.shape[1], 1))
x = Conv1D(96, kernel_size=3, activation='relu')(input_layer)
x = MaxPooling1D(pool_size=2)(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = attention_layer(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(96, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# === Train ===
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
train_start = datetime.now()
history = model.fit(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=100,
    batch_size=256,
    class_weight=class_weights,
   callbacks=[early_stop],
    verbose=1
)
train_end = datetime.now()
print(f" Total Training Time: {train_end - train_start}")
history_cnn = history


# === Plot Training & Validation Loss ===
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss', color='blue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange', linestyle='--', linewidth=2)
plt.title('Tuned CNN + Attention: Training vs Validation Loss', fontsize=14, weight='bold')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.7)
plt.tight_layout()
plt.savefig("cnn_bilstm_loss_plot.png", dpi=300)
plt.show()


# === Evaluate ===
test_start = datetime.now()
loss, acc = model.evaluate(X_test, y_test_cat)
test_end = datetime.now()
print(f"Total Testing Time: {test_end - test_start}")
print(f"\n Test Accuracy: {acc:.4f}")
print(f" Loss: {loss:.4f}")


# === Measure latency on a sample batch ===
batch_size_latency = 1000  
X_latency_sample = X_test[:batch_size_latency]

_ = model.predict(X_latency_sample)

# Measure inference time
start_time = time.time()
_ = model.predict(X_latency_sample)
end_time = time.time()

# Calculate total and per-sample latency
total_latency = end_time - start_time
average_latency_ms = (total_latency / batch_size_latency) * 1000

print(f"\n Inference Time for {batch_size_latency} samples: {total_latency:.4f} seconds")
print(f" Average Latency per Sample: {average_latency_ms:.4f} ms")

# === Predictions & Report ===
y_pred = model.predict(X_test).argmax(axis=1)
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']))

# === Confusion Matrix ===
cm = confusion_matrix(y_test, y_pred)
labels = ['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title("Confusion Matrix - Tuned CNN + Attention Model")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()

# === TPR and TNR Calculation ===
labels = ['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']
TPR = []
TNR = []

for i in range(len(cm)):
    tp = cm[i, i]
    fn = cm[i, :].sum() - tp
    fp = cm[:, i].sum() - tp
    tn = cm.sum() - (tp + fn + fp)

    tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
    tnr = tn / (tn + fp) if (tn + fp) != 0 else 0

    TPR.append(tpr)
    TNR.append(tnr)

# === Print TPR and TNR per class ===
print("\n True Positive Rate (Recall) and True Negative Rate (Specificity):")
for i, cls in enumerate(labels):
    print(f"{cls}:\n  TPR (Recall) = {TPR[i]:.4f}\n  TNR (Specificity) = {TNR[i]:.4f}")



# === TPR and FPR Calculation ===
labels = ['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']
TPR = []
FPR = []

for i in range(len(cm)):
    tp = cm[i, i]
    fn = cm[i, :].sum() - tp
    fp = cm[:, i].sum() - tp
    tn = cm.sum() - (tp + fn + fp)

    tpr = tp / (tp + fn) if (tp + fn) != 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) != 0 else 0

    TPR.append(tpr)
    FPR.append(fpr)

# Print Results
print("\n TPR (Recall) and FPR (Fall-out) per class:")
for i, cls in enumerate(labels):
    print(f"{cls}:\n   TPR (Recall) = {TPR[i]:.4f}\n   FPR = {FPR[i]:.4f}")


## === AUROC Calculation ===    
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

# Binarize the test labels
y_test_bin = label_binarize(y_test, classes=[0, 1, 2, 3])

# Predict class probabilities
y_score = model.predict(X_test)

# Compute AUROC for each class
auroc_per_class = roc_auc_score(y_test_bin, y_score, average=None)

# Print AUROC for each class
class_names = ['AttackFree', 'DoS', 'Fuzzy', 'Impersonation']
print("\n AUROC Score per Class:")
for i, score in enumerate(auroc_per_class):
    print(f"{class_names[i]}: AUROC = {score:.4f}")


